title: What's the impact of the new Lisbon eligibility on Overdraft and Installment Loans users?
author: Helder Silva 
date: 2022-04-08
region: EU
tags: installment loans, overdraft, bank products, eligibility, credit risk, lisbon, einsteinium
summary: With the credit score upgrade to Lisbon 2.0, we can see that there was a constant increase of eligibility across all markets and products. This increase was particularly visible for Overdraft MAUs in Germany (168k extra MAUs eligible in March), and TBIL MAUs in France (with 213k more eligible MAUs). On the other hand, this eligibility change leads to about 22k overdraft users across Germany and Austria no longer being eligible for these products, as well as 2.4k Installment Loans users across all 4 eligible markets.

![header](lisbon_impact_header.png)

### Intro
Lisbon is a Data Science microservice that calculates Probability of Default (PD) at N26. In the beginning of April 2022, we went live with version 2.0 of this service, which came with a major change to Lisbon Codebase - you can find more information on this change [here](https://number26-jira.atlassian.net/wiki/spaces/DATA/pages/2647818474/LS+2.0+Release+Notes).

These changes mean that the way we calculate the probability of default are now different after this release, leading to users that weren't eligible in the past for Overdraft and Transaction Based Installment Loans (also mentioned as TBIL or Installment Loans) are now eligible, and vice-versa. Therefore, this deep-dive aims to evaluate these eligibility changes by comparing users eligible at the end of February 2022 (with the old version of Lisbon), and end of March 2022 (taking all eligibility parameters as of this date, and new Lisbon scores as of April 4th).

The eligibility criteria used in this research were:

#### Overdraft Eligibility Criteria 

Tracked:
-  their calculated internal credit score is 12 or below
- the accepted T&C country is Germany or Austria
- age is not less than 18
- don't have a seized account
- customer doesn't have 2 or more direct debit reversal, due to insufficient funds, within the last 3 months
- no ongoing dunning process for this customer

Not Tracked:
- customer is not receiving unemployment or similar social benefit
- customer is not receiving substitute payments in case of bankrupt employers 



#### Installment Loans Eligibility Criteria

Tracked:
- has an eligible Lisbon score (in Einsteinium), up to 12 in DEU, ESP and ITA, up to 9 in FRA
- has German T&Cs (adding specific details for other markets)
- doesn't have an on-going dunning process
- haven't had an installment loan written-off


### Below we will be answering the following questions:
- [How many users are eligible before and after the Lisbon upgrade?](#section1)
- [How did the changes impact users' eligibility?](#section2)
- [How many no longer eligible users are using our products?](#section3)
- [What is the credit score distribution of the users who are no longer eligible?](#section4)

### Our main insights are:
- With the credit score upgrade to Lisbon 2.0, we can see that there was a constant increase of eligibility across all markets and products. This increase was particularly visible for Overdraft MAUs in Germany (168k extra MAUs eligible in March), and TBIL MAUs in France (with 213k more eligible MAUs).
- On the other hand, this eligibility change leads to about 22k overdraft users across Germany and Austria no longer being eligible for these products, as well as 2.4k Installment Loans users across all 4 eligible markets.

In [1]:
%%capture
cd/app/

In [2]:
%%capture
!pip install duckdb
!pip install altair

In [3]:
import pandas as pd
from utils.datalib_database import df_from_sql
import utils.altair_functions as af
import duckdb

con = duckdb.connect(database=":memory:", read_only=False)

In [4]:
feb_df = df_from_sql(
    "redshiftreader",
    "research/product/bank_products/20220330_lisbon_impact_on_bank_products/feb_eligibility_users.sql",
)

In [5]:
mar_df = df_from_sql(
    "redshiftreader",
    "research/product/bank_products/20220330_lisbon_impact_on_bank_products/mar_eligibility_users.sql",
)

In [6]:
users_df = df_from_sql(
    "redshiftreader",
    "research/product/bank_products/20220330_lisbon_impact_on_bank_products/tbil_od_users.sql",
)

<a id='section1'></a>
# How many users are eligible before and after the Lisbon upgrade?

Here we can see that for both Overdraft and Installment Loans users we see an increase of the number of eligible users between February and March. Also, we can see that the German market consistently has more eligible users than all other markets combined.

In [26]:
elig_users_query = """
select 
end_time,
case when label = 'Overdraft Eligible Users' then 'Overdraft' else 'TBIL' end 
|| ' February Eligible Users' as user_split,
tnc_country_group,
count(*) as n_users
from feb_df
group by 1, 2, 3
union all
select 
end_time,
case when label = 'Overdraft Eligible Users' then 'Overdraft' else 'TBIL' end 
|| ' March Eligible Users' as user_split,
tnc_country_group,
count(*) as n_users
from mar_df
where end_time = '2022-03-31'
group by 1, 2, 3
"""
elig_users_df = con.execute(elig_users_query).fetchdf()

In [9]:
af.column_multi(
    elig_users_df,
    "user_split",
    "tnc_country_group",
    "n_users",
    "user_split",
    150,
    500,
    "y",
)

alt.Chart(...)

Most of our product KPIs are based on the number of eligible MAUs, so we focus on those in the charts below. Once again, Germany is the strongest market, followed by France: 
- At the end of March, we had about 717k MAUs in Germany, therefore the percentage of eligible MAUs for Overdraft and Installment Loans are 82% and 85% respectively. 
- As for France, we had about 454k eligible users at the end of March, leading to about 55% of eligible Installment Loan users in this market.

In [27]:
elig_maus_query = """
select 
end_time,
case when label = 'Overdraft Eligible Users' then 'Overdraft' else 'TBIL' end 
|| ' February Eligible MAUs' as user_split,
tnc_country_group,
count(case when is_mau then 1 end) as n_maus
from feb_df 
group by 1, 2, 3
union all
select 
end_time,
case when label = 'Overdraft Eligible Users' then 'Overdraft' else 'TBIL' end 
|| ' March Eligible MAUs' as user_split,
tnc_country_group,
count(case when is_mau then 1 end) as n_maus
from mar_df 
where end_time = '2022-03-31'
group by 1, 2, 3
order by 1, 2, 3
"""
elig_maus_df = con.execute(elig_maus_query).fetchdf()

In [11]:
af.column_multi(
    elig_maus_df,
    "user_split",
    "tnc_country_group",
    "n_maus",
    "user_split",
    150,
    500,
    "y",
)

alt.Chart(...)

In the next chart you can find the difference of users and MAUs between February and March. We can see that there was a constant increase of eligibility across all markets and products. This increase was particularly visible for Overdraft MAUs in Germany (168k extra MAUs eligible in March), and TBIL MAUs in France (with 213k more eligible MAUs).

In [32]:
diff_query = """
with feb_elig as (
select 
case when label = 'Overdraft Eligible Users' then 'Overdraft' else 'TBIL' end as user_split,
tnc_country_group,
count(*) as n_users_feb,
count(case when is_mau then 1 end) as n_maus_feb
from feb_df 
group by 1, 2
),
mar_elig as (
select 
case when label = 'Overdraft Eligible Users' then 'Overdraft' else 'TBIL' end as user_split,
tnc_country_group,
count(*) as n_users_mar,
count(case when is_mau then 1 end) as n_maus_mar
from mar_df 
where end_time = '2022-03-31'
group by 1, 2
)
select 
user_split || ' User Diff' as user_split,
tnc_country_group, 
n_users_mar - n_users_feb as n_user_diff
from feb_elig
inner join mar_elig using (tnc_country_group, user_split)
union all 
select
user_split || ' MAU Diff' as user_split,
tnc_country_group, 
n_users_mar - n_maus_feb as n_mau_diff
from feb_elig
inner join mar_elig using (tnc_country_group, user_split)
order by 1, 2, 3
"""
diff_df = con.execute(diff_query).fetchdf()

In [13]:
af.column_multi(
    diff_df,
    "user_split",
    "tnc_country_group",
    "n_user_diff",
    "user_split",
    150,
    500,
    "y",
)

alt.Chart(...)

<a id='section2'></a>
# How did the changes impact users' eligibility?

Here we'll be focusing on overdraft Eligible MAUs, and check how their eligibility changed between February and March.

In [30]:
elig_maus_query = """
with feb_maus as(
select 
label,
user_id,
tnc_country_group
from feb_df 
where is_mau
group by 1, 2, 3
),
march_maus as (
select 
label,
user_id,
tnc_country_group
from mar_df 
where end_time = '2022-03-31'
and is_mau
group by 1, 2, 3
), 
totals as (
select
label,
tnc_country_group, 
case when f.user_id is not null and m.user_id is null then 'Eligible in Feb and not eligible in Mar'
when f.user_id is null and m.user_id is not null then 'Not eligible in Feb and eligible in Mar'
when f.user_id is not null and m.user_id is not null then 'Eligible in both Feb and Mar'
end as elig_split,
count(*) as n_users
from feb_maus f
full outer join march_maus m using (user_id, tnc_country_group, label)
group by 1, 2, 3
)
select *,
round(n_users::numeric /sum(n_users) over (partition by label, tnc_country_group), 3)*100 as perc_users
from totals where elig_split is not null
order by 1, 2, 3
"""
elig_maus_df = con.execute(elig_maus_query).fetchdf()
elig_maus_df[elig_maus_df["label"] == "Overdraft Eligible Users"]

,label,tnc_country_group,elig_split,n_users,perc_users
12,Overdraft Eligible Users,AUT,Eligible in Feb and not eligible in Mar,4824,6.8
13,Overdraft Eligible Users,AUT,Eligible in both Feb and Mar,49399,69.5
14,Overdraft Eligible Users,AUT,Not eligible in Feb and eligible in Mar,16817,23.7
15,Overdraft Eligible Users,DEU,Eligible in Feb and not eligible in Mar,33319,5.4
16,Overdraft Eligible Users,DEU,Eligible in both Feb and Mar,463507,74.8
17,Overdraft Eligible Users,DEU,Not eligible in Feb and eligible in Mar,122534,19.8


### Overdraft Eligible MAUs Split
We can see that for both Austria and Germany, the bulk of users were eligible for the overdraft product both in February and March. Only a small percentage lost eligibility between these 2 months (6.8% in Austria and 5.4% in Germany).

In [16]:
af.column_multi(
    elig_maus_df[elig_maus_df["label"] == "Overdraft Eligible Users"],
    "tnc_country_group",
    "elig_split",
    "perc_users",
    "tnc_country_group",
    200,
    500,
    "y",
)

alt.Chart(...)

### Installment Loans Eligible MAUs Split

And for Installment Loans we see a similar pattern, but higher percentages of users losing eligibility between February and March, reaching about 10% of all MAUs in Italy and Spain.

In [31]:
elig_maus_df[elig_maus_df["label"] == "Installment Loans Eligible Users"]

,label,tnc_country_group,elig_split,n_users,perc_users
0,Installment Loans Eligible Users,DEU,Eligible in Feb and not eligible in Mar,44838,6.9
1,Installment Loans Eligible Users,DEU,Eligible in both Feb and Mar,512225,78.3
2,Installment Loans Eligible Users,DEU,Not eligible in Feb and eligible in Mar,97172,14.9
3,Installment Loans Eligible Users,ESP,Eligible in Feb and not eligible in Mar,2899,10.2
4,Installment Loans Eligible Users,ESP,Eligible in both Feb and Mar,19174,67.8
5,Installment Loans Eligible Users,ESP,Not eligible in Feb and eligible in Mar,6217,22.0
6,Installment Loans Eligible Users,FRA,Eligible in Feb and not eligible in Mar,24009,8.8
7,Installment Loans Eligible Users,FRA,Eligible in both Feb and Mar,65320,23.8
8,Installment Loans Eligible Users,FRA,Not eligible in Feb and eligible in Mar,184858,67.4
9,Installment Loans Eligible Users,ITA,Eligible in Feb and not eligible in Mar,13122,10.1


In [17]:
af.column_multi(
    elig_maus_df[elig_maus_df["label"] == "Installment Loans Eligible Users"],
    "tnc_country_group",
    "elig_split",
    "perc_users",
    "tnc_country_group",
    200,
    500,
    "y",
)

alt.Chart(...)

<a id='section3'></a>
# How many no longer eligible users are using our products?

### Overdraft

This eligibility change leads to about 22k overdraft users across Germany and Austria no longer being eligible for these products:
- 7.6k users in dunning/ with DRs
- 13.1k od using users
- 1.1k od enabled users

In [18]:
od_elig_maus_query = """
with feb_users as(
select 
distinct user_id
from feb_df 
where label = 'Overdraft Eligible Users'
),
march_users as (
select 
distinct user_id
from mar_df 
where end_time = '2022-03-31'
and label = 'Overdraft Eligible Users'
), 
totals as (
select
case when has_dunning then '1. Currently in Dunning'
when has_write_off then '2. Has write-off'
when has_drs then '3. Has DRs'
when od_enabled and using_od then '4. Using Overdraft'
when od_enabled then '5. Overdraft Enabled Only' 
else 'Other'
end as od_split,
tnc_country_group, 
count(*) n_users,
count(case when f.user_id is null then 1 end) as not_elig_feb_users,
count(case when m.user_id is null then 1 end) as not_elig_mar_users,
count(case when f.user_id is not null then 1 end) as elig_feb_users,
count(case when m.user_id is not null then 1 end) as elig_mar_users
from users_df ud
left join feb_users f using (user_id)
left join march_users m using (user_id)
where (ud.od_enabled or ud.using_od)
and trim(tnc_country_group) in ('DEU', 'AUT')
group by 1, 2
)
select 
od_split,
tnc_country_group,
n_users,
coalesce(round(elig_feb_users::numeric/ n_users, 3)*100, 0) as perc_elig_feb, 
coalesce(round(elig_mar_users::numeric/ n_users, 3)*100, 0) as perc_elig_mar,
not_elig_feb_users,
not_elig_mar_users
from totals
order by 1, 2, 3 

"""
od_elig_maus_df = con.execute(od_elig_maus_query).fetchdf()
od_elig_maus_df

,od_split,tnc_country_group,n_users,perc_elig_feb,perc_elig_mar,not_elig_feb_users,not_elig_mar_users
0,1. Currently in Dunning,AUT,72,13.9,0.0,62,72
1,1. Currently in Dunning,DEU,813,14.1,0.0,698,813
2,2. Has write-off,DEU,24,0.0,0.0,24,24
3,3. Has DRs,AUT,377,43.0,37.4,215,236
4,3. Has DRs,DEU,9615,39.1,32.3,5859,6510
5,4. Using Overdraft,AUT,3568,77.0,61.6,822,1369
6,4. Using Overdraft,DEU,39065,82.4,70.0,6880,11733
7,5. Overdraft Enabled Only,AUT,6309,82.9,98.9,1079,71
8,5. Overdraft Enabled Only,DEU,78425,92.6,98.6,5811,1061


In [35]:
af.column_multi(
    od_elig_maus_df,
    "tnc_country_group",
    "od_split",
    "not_elig_mar_users",
    "tnc_country_group",
    200,
    500,
    "x",
)

alt.Chart(...)

### Installment Loans


The Lisbon 2.0 upgrade seems to impact about 2.4k Installment Loans users across all 4 eligible markets:
- 0.6k in dunning
- 1.8k currently using TBIL
- (on top of that, it would also impact another 2.7k users who had Installment Loans in the past and no longer have it)

In [21]:
tbil_elig_maus_query = """
with feb_users as(
select 
distinct user_id
from feb_df 
where label = 'Installment Loans Eligible Users'
),
march_users as (
select 
distinct user_id
from mar_df 
where end_time = '2022-03-31'
and label = 'Installment Loans Eligible Users'
), 
totals as (
select
case when has_dunning then '1. Currently in Dunning'
when has_write_off then '2. Has write-off'
when had_tbil_all_time and currently_using_tbil then '3. Currently Using TBIL'
when had_tbil_all_time then '4. Had TBIL in the past' 
else 'Other'
end as od_split,
tnc_country_group, 
count(*) n_users,
count(case when f.user_id is null then 1 end) as not_elig_feb_users,
count(case when m.user_id is null then 1 end) as not_elig_mar_users,
count(case when f.user_id is not null then 1 end) as elig_feb_users,
count(case when m.user_id is not null then 1 end) as elig_mar_users
from users_df ud
left join feb_users f using (user_id)
left join march_users m using (user_id)
where (had_tbil_all_time or currently_using_tbil)
and trim(tnc_country_group) in ('DEU', 'FRA', 'ITA', 'ESP')
group by 1, 2
)
select 
od_split,
tnc_country_group,
n_users,
coalesce(round(elig_feb_users::numeric/ n_users, 3)*100, 0) as perc_elig_feb, 
coalesce(round(elig_mar_users::numeric/ n_users, 3)*100, 0) as perc_elig_mar,
not_elig_feb_users,
not_elig_mar_users
from totals
order by 1, 2, 3 
"""
tbil_elig_maus_df = con.execute(tbil_elig_maus_query).fetchdf()
tbil_elig_maus_df

,od_split,tnc_country_group,n_users,perc_elig_feb,perc_elig_mar,not_elig_feb_users,not_elig_mar_users
0,1. Currently in Dunning,DEU,322,57.8,0.0,136,322
1,1. Currently in Dunning,ESP,3,0.0,0.0,3,3
2,1. Currently in Dunning,FRA,109,4.6,0.0,104,109
3,1. Currently in Dunning,ITA,5,40.0,0.0,3,5
4,2. Has write-off,DEU,200,5.0,0.0,190,200
5,2. Has write-off,FRA,1,0.0,0.0,1,1
6,3. Currently Using TBIL,DEU,11564,98.5,87.9,170,1396
7,3. Currently Using TBIL,ESP,352,98.3,95.2,6,17
8,3. Currently Using TBIL,FRA,1933,30.9,81.3,1336,361
9,3. Currently Using TBIL,ITA,610,94.8,98.5,32,9


In [36]:
af.column_multi(
    tbil_elig_maus_df,
    "tnc_country_group",
    "od_split",
    "not_elig_mar_users",
    "tnc_country_group",
    200,
    500,
    "x",
)

alt.Chart(...)

<a id='section4'></a>
# What is the credit score distribution of the users who are no longer eligible?

### Einsteinium vs Lisbon Scores

Overdraft and Installment Loans eligibility is calculated by the microservice Einsteinium. This service receives the credit scores from Lisbon and filters for eligible scores only, and therefore we used Einsteinium scores for this deep-dive since these are the ones actually used when users apply for these products.

For the exercise below, we will be taking Lisbon scores directy, since we will also need to look into the scores of non-eligible users. While doing so, we found some small discrepancies when comparing eligible users in Lisbon and in Einsteinium (we have a mismatch of up to 4% depending on the product). At the time of this research, these were being investigated and reconciled.

In [37]:
ls_es_df = df_from_sql(
    "redshiftreader",
    "research/product/bank_products/20220330_lisbon_impact_on_bank_products/lisbon_vs_einsteinium.sql",
)

In [38]:
ls_es_df

,product,ls_status,es_status,n_users,perc_users
0,Overdraft,Not Missing in Lisbon,Not Missing in Einsteinium,1341927,96.0
1,Overdraft,Not Missing in Lisbon,Missing in Einsteinium,27649,2.0
2,Overdraft,Missing in Lisbon,Not Missing in Einsteinium,27620,2.0
3,TBIL,Not Missing in Lisbon,Not Missing in Einsteinium,1367283,98.0
4,TBIL,Missing in Lisbon,Not Missing in Einsteinium,25161,1.8
5,TBIL,Not Missing in Lisbon,Missing in Einsteinium,2293,0.2


### Overdraft
As expected, the vast majority of non-eligible users tend to be more highly centered right after the cut-off eligibility score (12). We can also see some users that were considered non-eligible, but would potentially have eligible scores. These are mainly due to the issue identified above (especially the score 20, created to identify missing scores in Lisbon), as well as some of the eligibility criteria applied to this product.

In [19]:
od_score_query = """
with march_users as (
select 
distinct user_id
from mar_df 
where end_time = '2022-03-31'
and label = 'Overdraft Eligible Users'
)
select
case when od_enabled and using_od then 'Using Overdraft'
when od_enabled then 'Overdraft Enabled Only' 
else 'Other'
end as od_split,
coalesce(rating_class::int, 20) as rating_class,
count(*) n_users
from users_df ud
left join march_users m using (user_id)
where (ud.od_enabled or ud.using_od)
and trim(tnc_country_group) in ('DEU', 'AUT')
and not has_dunning
and not has_write_off
and not has_drs
and m.user_id is null
group by 1, 2
"""
od_score_df = con.execute(od_score_query).fetchdf()

In [20]:
af.column_multi(
    od_score_df, "od_split", "rating_class:O", "n_users", "od_split", 300, 500, "x"
)

alt.Chart(...)

In [22]:
tbil_score_query = """
with march_users as (
select 
distinct user_id
from mar_df 
where end_time = '2022-03-31'
and label = 'Installment Loans Eligible Users'
)
select
case when had_tbil_all_time and currently_using_tbil then 'Currently Using TBIL'
when had_tbil_all_time then 'Had TBIL in the past' 
else 'Other'
end as od_split,
coalesce(rating_class::int, 20) as rating_class,
count(*) n_users
from users_df ud
left join march_users m using (user_id)
where (had_tbil_all_time or currently_using_tbil)
and trim(tnc_country_group) in ('DEU', 'FRA', 'ITA', 'ESP')
and not has_dunning
and not has_write_off
and not has_drs
and m.user_id is null
group by 1, 2
"""
tbil_score_df = con.execute(tbil_score_query).fetchdf()

### Installment Loans

We can see a similar pattern, the main difference being that here we see more credit scores between 10 and 12, which is to be expected, since in the French market we only consider valid scores up to 9.

In [23]:
af.column_multi(
    tbil_score_df, "od_split", "rating_class:O", "n_users", "od_split", 300, 500, "x"
)

alt.Chart(...)